# API Sports Connection

Notebook to read in different types of football data sources from the football api to understand date and stastical limitations.

In [116]:
import pandas as pd
import requests
import json
pd.set_option("display.max_columns", None)  # Show all columns
pd.set_option("display.max_rows", 1000)  # Show all columns


In [5]:
europe_league_ids = [2,3,848,527]
english_league_ids = [39,40,41,42,43,45,48,49,50,51,529,984,985, 1156]
all_leagues = europe_league_ids + english_league_ids
seasons = [2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025]

In [6]:
API_KEY = "ad4b0c261d9bb7ea250891693538fa3b"
HEADERS = {"x-apisports-key": API_KEY}

## Fixtures

Fixtures data is great. Appears to have all leagues and seasons back to 2010. Will easily be able to use this for training and generating features.

In [7]:
def get_league_fixtures(league, year):
    URL = "https://v3.football.api-sports.io/fixtures"

    params = {
        "league": league, 
        "season": year  
    }

    response = requests.get(url, headers=HEADERS, params=params)
    data = response.json()
    fixtures = data.get("response", [])
    match_list = []
    for match in fixtures:
        match_list.append({
            "fixture_id": match['fixture']['id'],
            "match_date": match['fixture']['date'],
            "league": match['league']['name'],
            "referee": match['fixture']['referee'],
            "season": match['league']['season'],
            "timezone": match['fixture']['timezone'],
            "ground": match['fixture']['venue']['name'],
            "city": match['fixture']['venue']['city'],
            "home_team": match['teams']['home']['name'],
            "away_team": match['teams']['away']['name'],
            "round": match['league']['round'],
            "status": match['fixture']['status']['short'],
            "home_win": match['teams']['home']['winner'],
            "away_win": match['teams']['away']['winner'],
            "home_score_90": match['goals']['home'],
            "away_score_90": match['goals']['away'],
            "home_score_45": match['score']['halftime']['home'],
            "away_score_45": match['score']['halftime']['away'],
            "home_score_et": match['score']['extratime']['home'],
            "away_score_et": match['score']['extratime']['away'],
            "home_score_pen": match['score']['penalty']['home'],
            "away_score_pen": match['score']['penalty']['away'],
            "venue_id": match['fixture']['venue']['id'],
            "league_id": match['league']['id'],
            "home_team_id": match['teams']['home']['id'],
            "away_team_id": match['teams']['away']['id'],
        })

    df = pd.DataFrame(match_list)
    return df

In [ ]:
all_fixtures_df = pd.DataFrame()
for league_id in english_league_ids + europe_league_ids:
    for season in seasons:
        df = get_league_fixtures(league_id, season)

        all_fixtures_df = pd.concat([all_fixtures_df, df], ignore_index=True)

all_fixtures_df.to_csv("../data/football_data/apisports_data.csv")


In [238]:
all_fixtures_df.groupby(['league'])['match_date'].max()

league
Championship                     2025-05-03T14:00:00+00:00
FA Cup                           2025-03-01T15:00:00+00:00
League Cup                       2025-02-06T20:00:00+00:00
League One                       2025-05-03T14:00:00+00:00
League Two                       2025-05-03T14:00:00+00:00
Premier League                   2025-05-25T15:00:00+00:00
Super Cup                        2025-01-25T21:00:00+00:00
UEFA Champions League            2025-02-19T20:00:00+00:00
UEFA Europa Conference League    2025-02-20T20:00:00+00:00
UEFA Europa League               2025-02-20T20:00:00+00:00
Name: match_date, dtype: object

## Injuries

In [156]:
def get_injuries_for_team(team_id, league_id, season):
    url = f"https://v3.football.api-sports.io/injuries?season={season}&league={league_id}&team={team_id}"
    response = requests.get(url, headers=HEADERS)
    data = response.json()
    
    return data['response']

def collect_injury_data(team_ids, league_id, season):
    injury_data = []

    for team_id in team_ids:
        injuries = get_injuries_for_team(team_id, league_id, season)
        
        for injury in injuries:
            player = injury['player']['name']
            player_id = injury['player']['id']
            game_type = injury['player']['type']
            injury_type = injury['player']['reason']
            date = injury['fixture']['date']
            fixture_id = injury['fixture']['id']
            
            injury_data.append({
                'player': player,
                'game_type': game_type,
                'injury_type': injury_type,
                'date': date,
                'season': season,
                'fixture_id': fixture_id,
                'player_id': player_id,
                'team_id': team_id,
            })

    return pd.DataFrame(injury_data)

In [81]:
LEAGUE_ID = 39  # Example: Premier League (Check API-Football docs for other league IDs)
SEASON = 2023  # Specify the season

# API Endpoint
url = "https://v3.football.api-sports.io/injuries"


# Query parameters
params = {
    "league": LEAGUE_ID,
    "season": SEASON
}

# Make the request
response = requests.get(url, headers=HEADERS, params=params)
    data = response.json()
    injuries = data.get("response", [])
else:
    print("Error:", response.json())

In [85]:
df = pd.json_normalize(injuries)
df.columns = [col.replace('.', '_') for col in df.columns]
df

,player_id,player_name,player_photo,player_type,player_reason,team_id,team_name,team_logo,fixture_id,fixture_timezone,fixture_date,fixture_timestamp,league_id,league_season,league_name,league_country,league_logo,league_flag
0,18957,M. Obafemi,https://media.api-sports.io/football/players/1...,Missing Fixture,Thigh Injury,44,Burnley,https://media.api-sports.io/football/teams/44.png,1035037,UTC,2023-08-11T19:00:00+00:00,1691780400,39,2023,Premier League,England,https://media.api-sports.io/football/leagues/3...,https://media.api-sports.io/flags/gb-eng.svg
1,1452,M. Elneny,https://media.api-sports.io/football/players/1...,Missing Fixture,Knock,42,Arsenal,https://media.api-sports.io/football/teams/42.png,1035038,UTC,2023-08-12T11:30:00+00:00,1691839800,39,2023,Premier League,England,https://media.api-sports.io/football/leagues/3...,https://media.api-sports.io/flags/gb-eng.svg
2,643,Gabriel Jesus,https://media.api-sports.io/football/players/6...,Missing Fixture,Knee Injury,42,Arsenal,https://media.api-sports.io/football/teams/42.png,1035038,UTC,2023-08-12T11:30:00+00:00,1691839800,39,2023,Premier League,England,https://media.api-sports.io/football/leagues/3...,https://media.api-sports.io/flags/gb-eng.svg
3,138835,F. Balogun,https://media.api-sports.io/football/players/1...,Questionable,Ankle Injury,42,Arsenal,https://media.api-sports.io/football/teams/42.png,1035038,UTC,2023-08-12T11:30:00+00:00,1691839800,39,2023,Premier League,England,https://media.api-sports.io/football/leagues/3...,https://media.api-sports.io/flags/gb-eng.svg
4,1427,A. S. Lokonga,https://media.api-sports.io/football/players/1...,Questionable,Muscle Injury,42,Arsenal,https://media.api-sports.io/football/teams/42.png,1035038,UTC,2023-08-12T11:30:00+00:00,1691839800,39,2023,Premier League,England,https://media.api-sports.io/football/leagues/3...,https://media.api-sports.io/flags/gb-eng.svg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3848,2413,Richarlison,https://media.api-sports.io/football/players/2...,Missing Fixture,Calf Injury,47,Tottenham,https://media.api-sports.io/football/teams/47.png,1035553,UTC,2024-05-19T15:00:00+00:00,1716130800,39,2023,Premier League,England,https://media.api-sports.io/football/leagues/3...,https://media.api-sports.io/flags/gb-eng.svg
3849,19032,R. Sessegnon,https://media.api-sports.io/football/players/1...,Missing Fixture,Thigh Injury,47,Tottenham,https://media.api-sports.io/football/teams/47.png,1035553,UTC,2024-05-19T15:00:00+00:00,1716130800,39,2023,Premier League,England,https://media.api-sports.io/football/leagues/3...,https://media.api-sports.io/flags/gb-eng.svg
3850,697,M. Solomon,https://media.api-sports.io/football/players/6...,Missing Fixture,Knee Injury,47,Tottenham,https://media.api-sports.io/football/teams/47.png,1035553,UTC,2024-05-19T15:00:00+00:00,1716130800,39,2023,Premier League,England,https://media.api-sports.io/football/leagues/3...,https://media.api-sports.io/flags/gb-eng.svg
3851,204039,D. Udogie,https://media.api-sports.io/football/players/2...,Missing Fixture,Thigh Injury,47,Tottenham,https://media.api-sports.io/football/teams/47.png,1035553,UTC,2024-05-19T15:00:00+00:00,1716130800,39,2023,Premier League,England,https://media.api-sports.io/football/leagues/3...,https://media.api-sports.io/flags/gb-eng.svg


In [210]:
injuries = get_injuries_for_team(team_ids[0], league_id, 2024)

In [158]:
injuries = collect_injury_data(team_ids, english_league_ids[0], seasons[0])

In [175]:
injuries[injuries['player'] == 'M. Rashford']

,player,game_type,injury_type,date,season,fixture_id,player_id,team_id
1,M. Rashford,Missing Fixture,Shoulder Injury,2021-08-22T13:00:00+00:00,2021,710573,909,33
8,M. Rashford,Missing Fixture,Shoulder Injury,2021-08-29T15:30:00+00:00,2021,710585,909,33
13,M. Rashford,Missing Fixture,Shoulder Injury,2021-09-11T14:00:00+00:00,2021,710593,909,33
20,M. Rashford,Missing Fixture,Shoulder Injury,2021-09-19T13:00:00+00:00,2021,710604,909,33
23,M. Rashford,Missing Fixture,Shoulder Injury,2021-09-25T11:30:00+00:00,2021,710613,909,33
147,M. Rashford,Missing Fixture,Illness,2022-05-07T16:30:00+00:00,2021,710908,909,33
154,M. Rashford,Questionable,Illness,2022-05-22T15:00:00+00:00,2021,710931,909,33


## Coaches 

In [215]:
def get_managers(team_id):
    url = f"https://v3.football.api-sports.io/coachs?team={team_id}"
    response = requests.get(url, headers=HEADERS).json()
    
    managers = []
    for coach in response['response']:
        managers.append({
            'Team': coach['team']['name'] if coach.get('team') else 'Unknown',
            'Manager': coach['name'],
            'Nationality': coach['nationality'],
            'Start Date': coach.get('start'),
            'End Date': coach.get('end', 'Present')
        })
    return managers

In [216]:
get_managers(33)

[{'Team': 'Manchester United',
  'Manager': 'E. ten Hag',
  'Nationality': 'Netherlands',
  'Start Date': None,
  'End Date': 'Present'},
 {'Team': 'Manchester United',
  'Manager': 'Ruben Amorim',
  'Nationality': 'Portugal',
  'Start Date': None,
  'End Date': 'Present'}]

## Statistics

In [77]:
    BASE_URL = "https://v3.football.api-sports.io"

    url = f"{BASE_URL}/fixtures/statistics?fixture={868324}"
    response = requests.get(url, headers=HEADERS)
    data = response.json()
    fixtures = data.get("response", [])
    df = pd.json_normalize(fixtures)
    df.columns = [col.replace('.', '_') for col in df.columns]
    df = df.explode("statistics")
    df_stats = pd.json_normalize(df["statistics"])
    df = df.drop(columns=["statistics"]).reset_index(drop=True)
    df = pd.concat([df, df_stats], axis=1)
    df = df.pivot(index=["team_id", "team_name"], columns="type", values="value").reset_index()

In [79]:
df.columns = df.columns.str.lower().str.replace(' ', '_')


In [80]:
df

type,team_id,team_name,ball_possession,blocked_shots,corner_kicks,fouls,goalkeeper_saves,offsides,passes_%,passes_accurate,red_cards,shots_insidebox,shots_off_goal,shots_on_goal,shots_outsidebox,total_shots,total_passes,yellow_cards,expected_goals
0,33,Manchester United,53%,6,5,14,2,1,84%,415,None,12,7,8,9,21,493,1,2.89
1,36,Fulham,47%,1,4,10,6,1,84%,358,None,6,6,3,4,10,426,2,1.80


In [76]:
BASE_URL = "https://v3.football.api-sports.io"

url = f"{BASE_URL}/fixtures/statistics?fixture={868324}"
response = requests.get(url, headers=HEADERS)
data = response.json()

fixtures = data.get("response", [])
df = pd.json_normalize(fixtures)
df.columns = [col.replace('.', '_') for col in df.columns]
df = df.explode("statistics")
df_stats = pd.json_normalize(df["statistics"])
df = df.drop(columns=["statistics"]).reset_index(drop=True)
df = pd.concat([df, df_stats], axis=1)
df = df.pivot(index=["team_id", "team_name"], columns="type", values="value").reset_index()
df

# if response.status_code == 200:
#     stats = response.json()["response"]
#    print({team["team"]["name"]: {stat["type"]: stat["value"] for stat in team["statistics"]} for team in stats})



type,team_id,team_name,Ball Possession,Blocked Shots,Corner Kicks,Fouls,Goalkeeper Saves,Offsides,Passes %,Passes accurate,Red Cards,Shots insidebox,Shots off Goal,Shots on Goal,Shots outsidebox,Total Shots,Total passes,Yellow Cards,expected_goals
0,33,Manchester United,53%,6,5,14,2,1,84%,415,None,12,7,8,9,21,493,1,2.89
1,36,Fulham,47%,1,4,10,6,1,84%,358,None,6,6,3,4,10,426,2,1.80


In [73]:
fixtures = data.get("response", [])
df = pd.json_normalize(fixtures)
df.columns = [col.replace('.', '_') for col in df.columns]
df

,statistics,team_id,team_name,team_logo
0,"[{'type': 'Shots on Goal', 'value': 8}, {'type...",33,Manchester United,https://media.api-sports.io/football/teams/33.png
1,"[{'type': 'Shots on Goal', 'value': 3}, {'type...",36,Fulham,https://media.api-sports.io/football/teams/36.png


In [58]:
data = response.json()
stats = data.get("response", [])
stats = pd.json_normalize(stats)


In [59]:
stats

,statistics,team.id,team.name,team.logo
0,"[{'type': 'Shots on Goal', 'value': 8}, {'type...",33,Manchester United,https://media.api-sports.io/football/teams/33.png
1,"[{'type': 'Shots on Goal', 'value': 3}, {'type...",36,Fulham,https://media.api-sports.io/football/teams/36.png


## Predictions

In [101]:
url = "https://v3.football.api-sports.io/predictions"
params = {
    "fixture": 157391,
}

response = requests.get(url, headers=HEADERS, params=params)

data = response.json()
predictions = data.get("response", [])

# Convert to DataFrame using json_normalize
df = pd.json_normalize(predictions)
df.columns = [col.replace('.', '_') for col in df.columns]


In [122]:
df['h2h']


0    [{'fixture': {'id': 157060, 'referee': 'Kevin ...
Name: h2h, dtype: object